In [2]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import re
import math
import numpy
import pandas as pd 

import json
import csv
from itertools import chain
from typing import List
from multiprocessing.dummy import Pool as ThreadPool 


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
def make_df_with_float_salary(data):
    for i in range(len(data) - 1):
        data[i][2] = data[i][2].replace('от ', '').replace('до ', '').replace('.', '').split(' руб ')
    with_salary = [r for r in data if len(r[2]) == 2]
    with_float_salary = pd.DataFrame([[r[0], r[1], (1 if r[2][1] == 'на руки' else 0.8) * sum([int(el) for el in r[2][0].split(' ')]) / len(r[2][0].split(' ')) / 1000] for r in with_salary])
    with_float_salary = with_float_salary.sort_values(by=[2, 1])
    return with_float_salary

In [102]:
def make_df(city_abbr: str) -> pd.DataFrame:
    '''Returns a pd.DataFrame (vacancy url, adress, salary) of fulltime vacancies in 
       corresponding to the abbreviation city and export it to csv'''
    
    cities_dir = {'spb': 'Санкт-Петербург', 'smr': 'Самара', 'kzn': 'Казань', 'vdk': 'Владивосток'}
    
    with open(f'../hh-data/tables/{city_abbr}/fulltime/completed.csv') as file:
        data = [row.replace('\xa0', '').split('; ') for row in file.read().split('\n')]
    
    for i in range(len(data) - 1):
        data[i][2] = data[i][2].replace('от ', '').replace('до ', '').replace('.', '').split(' руб ')
    with_set_salary = [r for r in data if len(r[2]) == 2]
    df = pd.DataFrame([[r[0], 
                        f'{cities_dir[city_abbr]}, ' * (cities_dir[city_abbr] not in r[1]) + r[1], 
                        int((1 if r[2][1] == 'на руки' else 0.8) * sum([int(el) for el in r[2][0].split(' ')]) / len(r[2][0].split(' ')) / 1000)] for r in with_set_salary])
    df = df.sort_values(by=[2, 1])
    df.columns = ['url', 'adress', 'salary']
    df = df[df.salary > 0][df.salary <= 300]    
    df.to_csv(f'../hh-data/tables/{city_abbr}/fulltime/url-adress-salary.csv', index=False)
    return df

In [103]:
def plot_salaries(df, n=20):
    n, bins, patches = plt.hist(list(df['salary'])[20:], n, density=True, facecolor='g', alpha=0.75)

    plt.xlabel('Gpm')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.axis([10, 300, 0, 0.03])
    plt.rcParams["figure.figsize"] = [11, 11]
    plt.show()


In [104]:
def plot_ln_of_salaries(df, n=20):
    n, bins, patches = plt.hist([math.log(a) for a in list(df['salary'])[20:]], n, density=True, facecolor='g', alpha=0.75)

    plt.xlabel('Gpm')
    plt.ylabel('Probability')
    plt.grid(True)

    plt.rcParams["figure.figsize"] = [11, 11]
    plt.show()


In [108]:
spb_df = make_df('spb')
smr_df = make_df('smr')
kzn_df = make_df('kzn')


/home/disa/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
